# Messing with embedding spaces

This is complimentary material to my blog post about embedding spaces in the context of a Kaggle competition, where the goal is to guess the original prompt that was given to the LLM to convert original text A to text B.

Here we will look into the scoring mechanism only.

In [2]:
from sentence_transformers import SentenceTransformer


D:\anaconda-ozy\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Sentence we are trying to guess
ORIGINAL = "Convert this text into a sea shanty."


candidates = [
    "Convert this text into a shanty.",
    "Convert this text into a song.",
    "Convert this text into song lyrics.",
    "Convert this text into rap format.",
    "Convert this text into lyrics.",
    "Transform this text into a sea shanty.",
    "Make this text better.",
    "Improve this text."
]

In [4]:
model = SentenceTransformer('sentence-transformers/sentence-t5-base')

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

def score_sentences(original, sentences, scoring_func=model.encode):
    """The scoring function is cubic cosine similarity."""
    
    embeddings = scoring_func([original]+sentences)
    score = cosine_similarity(embeddings)[0]**3
    return score[1:]

In [6]:
scores = score_sentences(ORIGINAL, candidates)



In [47]:
import pandas as pd

def scores_to_df(candidates,scores):
    scores_df = pd.Series(dict(zip(candidates,scores))).reset_index()
    scores_df.columns = ["sentence", "score"]
    return scores_df.sort_values(by="score", ascending=False)
    
    

In [48]:
scores_to_df(candidates,scores)

,sentence,score
5,Transform this text into a sea shanty.,0.670905
1,Convert this text into a song.,0.658017
2,Convert this text into song lyrics.,0.655832
6,Make this text better.,0.649546
7,Improve this text.,0.640520
4,Convert this text into lyrics.,0.636384
3,Convert this text into rap format.,0.620373
0,Convert this text into a shanty.,0.606098


Two interesting things:

- The scoring (embedding+cosine) is quite unforgiving. It drops sharply when you don't guess the exact words, even if you guess in the same ballpark.
- "Make this text better", already beats most of the song-related prompts.

Now let's add more variation to our sentence pool, then check: out of all sentences in our sentence pool, which one has, on average, highest similarity to the whole group?

In [9]:
sentence_pool = [
    ORIGINAL,
    "Translate this text into Chinese.",
    "Make this text ELI5.",
    "Convert this text into a Haiku.",
    "Turn this text into a newspaper article.",
    "Make this text sound more assertive.",
    "Translate this text into Pig Latin.",
    "Rewrite this text from the point of view of a medieval scholar.",
    "Rewrite this text without adjectives.",
    "Make this text shorter.",
    "Fluff up this text.",
    "Make this text better.",
    "Drop all the vowels from this text.",
    "Reimagine this text as a school principal's speech.",
    "Write this text as a tweet.",
    "Convert this text into an elevator pitch.",
    "Transform this text into a paper abstract.",
    "Rewrite this text in all caps.",
    "Turn this text into a fable.",
    "Rewrite this text as if it's a Bible verse.",
    "Paraphrase this text while retaining the original meaning."
] 

In [10]:
import numpy as np

embeddings = model.encode(sentence_pool)
score = cosine_similarity(embeddings)**3
score_mean = np.mean(score, 1)

score_mean_table = pd.Series(dict(zip(sentence_pool,score_mean))).reset_index()
score_mean_table.columns = ["sentence", "mean score"]
score_mean_table.sort_values(by="mean score", ascending=False)

,sentence,mean score
11,Make this text better.,0.675470
10,Fluff up this text.,0.649809
4,Turn this text into a newspaper article.,0.634063
19,Rewrite this text as if it's a Bible verse.,0.626964
9,Make this text shorter.,0.622745
15,Convert this text into an elevator pitch.,0.619233
5,Make this text sound more assertive.,0.615879
16,Transform this text into a paper abstract.,0.615197
14,Write this text as a tweet.,0.615048
18,Turn this text into a fable.,0.608951


Even in a more varied pool, "Make this text better." highest on average! That means, we can call this the "mean" sentence -- it vague and general enough, and its embedding is positioned centrally in relation to the others in the embedding space. Therefore, it can be our default guess when trying to guess any sentence in this pool. Especially since the scoring is, as seen before, quite unforgiving, and a mean sentence beats one that feels subjectively closer to the theme.

## Automated generation of mean sentence

Can we do better than this? What if we searched the token space to find the sentence that gets the highest score in a given pool?

In [11]:
tokenizer = model.tokenizer

# Get set of all special token IDs
special_token_ids = set(tokenizer.all_special_ids)

# Filter out special tokens
vocabulary = [token for token, token_id in tokenizer.vocab.items() if token_id not in special_token_ids]

len(vocabulary)

31997

In [12]:
" "  in vocabulary

False

In [13]:
def mean_score(orig_sentences, new_sentences):
    embeddings_orig = model.encode(orig_sentences, batch_size=256)
    embeddings_new = model.encode(new_sentences, batch_size=256)
    score = cosine_similarity(embeddings_orig,embeddings_new)**3
    score_mean = np.mean(score, 0)
    return score_mean

Here we do a simple greedy search that simply picks the next best token and appends it to the selected index, with two passes. First pass constructs the initial mean sentence and the second best refines it.

This is just to demonstrate the point -- there are ways that can yield better results, such as beam search or even fancier methods like genetic algorithms.

In [30]:
current = [""]*20
current_text = ""

current_score = 0
for _ in range(2):
    for i in range(10):
        candidates = [current.copy() for _ in vocabulary]
        for j,t in enumerate(vocabulary):
            candidates[j][i] = t
            candidates[j] = [t for t in candidates[j] if t!=""]        

        # current_strip = [t for t in current if t!=""]
        candidate_texts = [tokenizer.convert_tokens_to_string(c) for c in candidates]

        # print(len(candidates))
        # print(candidates[0])

        scores = mean_score(sentence_pool, candidate_texts)

        current_score = np.max(scores)
        # best_idx = np.argsort(-scores)[:5]        

        next_token = vocabulary[np.argmax(scores)]
        current[i] = next_token
        current_text = tokenizer.convert_tokens_to_string(current)
        print(f"{current_text=} {current_score=}")


current_text='Text' current_score=0.5537645
current_text='Text thus' current_score=0.6248839
current_text='Text thus piece' current_score=0.6438213
current_text='Text thus pieceou' current_score=0.65894973
current_text='Text thus pieceouTHER' current_score=0.66563773
current_text='Text thus pieceouTHERT' current_score=0.666775
current_text='Text thus pieceouTHERTTION' current_score=0.66686046
current_text='Text thus pieceouTHERTTION ' current_score=0.66686046
current_text='Text thus pieceouTHERTTION lucrarea' current_score=0.669691
current_text='Text thus pieceouTHERTTION lucrarea.' current_score=0.676661
current_text='Text thus pieceouTHERTTION lucrarea.' current_score=0.676661
current_text='Text this pieceouTHERTTION lucrarea.' current_score=0.69148684
current_text='Text this PieceouTHERTTION lucrarea.' current_score=0.69295275
current_text='Text this PieceICTHERTTION lucrarea.' current_score=0.6964444
current_text='Text this PieceICWLTTION lucrarea.' current_score=0.70065045
current

This sentence scores even higher than 'Make this text better' on average and it's nonsensical.


Going back to the initial sentence guessing problem - what if we appended our educated guesses with this mean sentence?

In [35]:
# Sentence we are trying to guess
ORIGINAL = "Convert this text into a sea shanty."
MEAN_SENTENCE = "Text this PieceICWLISHTION aslucrarea."

candidates = [
    "Convert this text into a shanty.",
    "Convert this text into a song.",
    "Convert this text into song lyrics.",
    "Convert this text into rap format.",
    "Convert this text into lyrics.",
    "Transform this text into a sea shanty.",
    "Make this text better.",
    "Improve this text."
]

In [52]:
augmented_candidates = [c+" "+MEAN_SENTENCE+" "+c for c in candidates]  + [MEAN_SENTENCE]
    

In [53]:
scores_normal = score_sentences(ORIGINAL, candidates)
scores_augmented = score_sentences(ORIGINAL, augmented_candidates)


In [54]:
scores_to_df(candidates,scores_normal)

,sentence,score
5,Transform this text into a sea shanty.,0.970698
0,Convert this text into a shanty.,0.901210
1,Convert this text into a song.,0.638192
6,Make this text better.,0.621427
4,Convert this text into lyrics.,0.619016
2,Convert this text into song lyrics.,0.609144
7,Improve this text.,0.604455
3,Convert this text into rap format.,0.593446


In [55]:
pd.options.display.max_colwidth = 100
scores_to_df(augmented_candidates,scores_augmented)

,sentence,score
5,Transform this text into a sea shanty. Text this PieceICWLISHTION aslucrarea.,0.919560
0,Convert this text into a shanty. Text this PieceICWLISHTION aslucrarea.,0.856438
8,Text this PieceICWLISHTION aslucrarea.,0.660113
6,Make this text better. Text this PieceICWLISHTION aslucrarea.,0.651418
3,Convert this text into rap format. Text this PieceICWLISHTION aslucrarea.,0.651195
1,Convert this text into a song. Text this PieceICWLISHTION aslucrarea.,0.647883
7,Improve this text. Text this PieceICWLISHTION aslucrarea.,0.645544
4,Convert this text into lyrics. Text this PieceICWLISHTION aslucrarea.,0.641856
2,Convert this text into song lyrics. Text this PieceICWLISHTION aslucrarea.,0.627022


When the exact words are guessed ("shanty"), prepending with the mean prompt harms. However, for every other guess